# Objetivo:

- Recuperação dos dados da Camada Raw;
- Salva de forma dinâmica as colunas no banco de dados.
- Subir as informações para o Banco de Dados na tabela 'stg_empresas_bronze' e 'stg_socios_bronze'

### Import das Bibliotecas

In [ ]:
import polars as pl
import os
import psycopg2
import gc
from sqlalchemy import create_engine
from dotenv import load_dotenv
from psycopg2 import sql

### Definição dos Diretórios

In [ ]:
DATA_DIR = '../data'

BRONZE_DIR = os.path.join(DATA_DIR, 'bronze')
RAW_DIR = os.path.join(DATA_DIR, 'raw')

# Garante que os diretórios existam
os.makedirs(BRONZE_DIR, exist_ok=True)
os.makedirs(RAW_DIR, exist_ok=True)

### Definição dos nomes dos Arquivos na camada Raw Dinâmico

In [ ]:
EMPRESA_FILE_RAW = None
SOCIO_FILE_RAW = None

# Lista todos os arquivos no diretório bronze

try:
    raw_files = os.listdir(RAW_DIR)

    # Procura pelos arquivos de Empresa e Socio com base no sufixo
    for filename in raw_files:
        if filename.endswith('.EMPRECSV'):
            EMPRESA_FILE_RAW = os.path.join(RAW_DIR, filename)
        elif filename.endswith('.SOCIOCSV'):
            SOCIO_FILE_RAW = os.path.join(RAW_DIR, filename)

    # Verifica se os arquivos foram encontrados
    if EMPRESA_FILE_RAW is None:
        raise FileNotFoundError(f"Nenhum arquivo terminando com '.EMPRECSV' encontrado em {RAW_DIR}. Verifique o nome do arquivo.")
    if SOCIO_FILE_RAW is None:
        raise FileNotFoundError(f"Nenhum arquivo terminando com '.SOCIOCSV' encontrado em {RAW_DIR}. Verifique o nome do arquivo.")

except FileNotFoundError as e:
    print(f"ERRO: {e}")
    exit()
except Exception as e:
    print(f"Ocorreu um erro ao listar arquivos no diretório raw: {e}")
    exit()

### Leitura da Raw

### Parâmetros de Leitura dos Arquivos

In [ ]:
# --- Parâmetros de Leitura dos Arquivos (Comum para Receita Federal) ---
# Os arquivos da Receita Federal geralmente são delimitados por ponto e vírgula,
# codificados em latin-1 e não possuem cabeçalho.
READ_PARAMS = {
    "separator": ";",
    "encoding": "latin1",
    "has_header": False,
    # infer_schema_length=0 => força todas como string sem precisar mapear coluna por coluna
    "infer_schema_length": 0 
}

#### Tabela Empresas

In [ ]:
# Ler tabela Empresas

df_empresas = pl.read_csv(EMPRESA_FILE_RAW, **READ_PARAMS)

# Salva CSV Temporário no Diretório Bronze

csv_empresas_temp = os.path.join(BRONZE_DIR, "bronze_empresas_temp.csv")
df_empresas.write_csv(csv_empresas_temp)

print("Salvo arquivo em ", csv_empresas_temp)

# Deleta o DataFrame
del df_empresas

# Força o garbage collector a liberar a memória
gc.collect()

print("Liberado a memória RAM")

#### Tabela Sócios

In [ ]:
# Ler tabela Sócios

df_socios = pl.read_csv(SOCIO_FILE_RAW, **READ_PARAMS)

# Salva CSV Temporário no Diretório Bronze

csv_socios_temp = os.path.join(BRONZE_DIR, "bronze_socios_temp.csv")
df_socios.write_csv(csv_socios_temp)

print("Salvo arquivo em ", csv_socios_temp)

# Deleta o DataFrame
del df_socios

# Força o garbage collector a liberar a memória
gc.collect()

print("Liberado a memória RAM")

### Salvar no Postgres

In [ ]:
# Carrega variáveis de ambiente (.env)
load_dotenv(dotenv_path="../.env")

DB_USER = os.getenv("DB_USER")
DB_PASS = os.getenv("DB_PASSWORD")
DB_NAME = os.getenv("DB_NAME")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")

#### Mapeamento dos tipos do Polars para o Postgres

In [ ]:
def inferir_tipo_postgres(dtype: pl.datatypes.DataType) -> str:
    """Mapeia tipos do Polars para tipos do Postgres."""
    if dtype == pl.Int64 or dtype == pl.Int32 or dtype == pl.UInt32 or dtype == pl.UInt64:
        return "BIGINT"
    elif dtype == pl.Float64 or dtype == pl.Float32:
        return "DOUBLE PRECISION"
    elif dtype == pl.Boolean:
        return "BOOLEAN"
    elif dtype == pl.Datetime or dtype == pl.Date:
        return "TIMESTAMP"
    else:
        return "TEXT"

#### Função escrever no banco de Dados

In [ ]:
def escrever_banco(csv_temp, nome_tabela,
                   dbname, user, password, host, port):
    # Lê apenas 100 linhas para inferir o schema
    df_sample = pl.read_csv(
        csv_temp,
        separator=",",
        encoding="utf-8",
        has_header=False,
        infer_schema_length=1000,
        n_rows=100
    )

    # Monta schema dinâmico
    colunas = []
    for col, dtype in zip(df_sample.columns, df_sample.dtypes):
        pg_tipo = inferir_tipo_postgres(dtype)
        colunas.append(f"{col} {pg_tipo}")

    schema_sql = ",\n    ".join(colunas)

    # Conecta no Postgres
    conn = psycopg2.connect(
        dbname=dbname,
        user=user,
        password=password,
        host=host,
        port=port
    )
    cur = conn.cursor()

    # Cria tabela com nome dinâmico
    cur.execute(sql.SQL("""
        DROP TABLE IF EXISTS {tabela};
        CREATE TABLE {tabela} (
            {schema}
        );
    """).format(
        tabela=sql.Identifier(nome_tabela),
        schema=sql.SQL(schema_sql)
    ))

    # COPY direto, sem abrir no Python
    with open(csv_temp, "r", encoding="utf-8") as f:
        cur.copy_expert(
            sql.SQL("COPY {tabela} FROM STDIN WITH CSV HEADER DELIMITER ','").format(
                tabela=sql.Identifier(nome_tabela)
            ),
            f
        )

    conn.commit()
    cur.close()
    conn.close()

    print(f"Tabela '{nome_tabela}' salva no Postgres via COPY")

    # Remove CSV temporário
    if os.path.exists(csv_temp):
        os.remove(csv_temp)
        print("Arquivo temporário removido:", csv_temp)

#### Chamar funções

In [ ]:
escrever_banco(
    csv_temp=csv_empresas_temp,
    nome_tabela="stg_empresas_bronze",
    dbname=DB_NAME,
    user=DB_USER,
    password=DB_PASS,
    host=DB_HOST,
    port=DB_PORT
)

In [ ]:
escrever_banco(
    csv_temp=csv_socios_temp,
    nome_tabela="stg_socios_bronze",
    dbname=DB_NAME,
    user=DB_USER,
    password=DB_PASS,
    host=DB_HOST,
    port=DB_PORT
)